In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import os
#rcParams['font.family'] = 'serif'

In [2]:
#path = "/home/esteban/Escritorio/Practica/Data/G13_millennium"  # Guo2013    (MPA)
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014  (Durham)
path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016     (Durham)
os.chdir( path )

In [3]:
x = np.load('x.npy')
y = np.load('y.npy')
z = np.load('z.npy')
sfr = np.load('sfr.npy')
stellarmass = np.load('stellarmass.npy')
# === Density selection ===
#   Fila    log(rho)
#    15      -2.5
#    10      -3.0
#     5      -3.5
# =========================
log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
rho, mass_cut, sfr_cut = log_rho[5], mass_cuts[5], sfr_cuts[5]

mask = stellarmass > mass_cut
x = x[mask]
y = y[mask]
z = z[mask]

In [4]:
len(x)

10639

In [5]:
def RR (r,n,dr):
    V_c = (4./3)*np.pi*((10**(r+dr))**3 - (10**r)**3)
    V_t = 500.**3
    return (V_c/V_t)*n**2

In [6]:
%%time
path = "/home/esteban/Escritorio/Practica/Codes/C_Python"
os.chdir( path )
import SPack

NBIN = 30
Dmin = -2  # Min separation (log(d)) [d] = Mpc
Dmax = 1.  # Max separation (log(d))
bin_width = (Dmax - Dmin)/NBIN

len_data = len(x)
DD_array = np.zeros(NBIN).astype('int32')

# ===================== C - MODULE =========================
SPack.xi(DD_array, x, y, z, len_data, Dmin, Dmax, NBIN)
# ==========================================================

#bins = np.linspace(Dmin, Dmax, NBIN, endpoint = False)   
bins = np.array([Dmin + bin_width*i for i in range(NBIN)]) + bin_width *.5
RR_array = np.array([RR(r, len_data, bin_width) for r in bins])

CPU times: user 4 s, sys: 0 ns, total: 4 s
Wall time: 4 s


In [7]:
logxi = np.log10((2*DD_array/RR_array)-1)

/home/esteban/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':


In [8]:
logxi

array([        nan,  5.75123271,  5.45123194,  4.67410302,  4.37409387,
        4.0740756 ,  4.37615395,  4.42833853,  3.17382127,  3.87405419,
        3.52822606,  3.45004943,  3.14989606,  2.87765698,  2.75355814,
        2.66423694,  2.38511049,  2.10417735,  1.9892268 ,  1.78614154,
        1.53474389,  1.30624845,  1.03248429,  0.84385089,  0.6406827 ,
        0.5565403 ,  0.30016196,  0.19949219, -0.1240708 , -0.34575069])

In [9]:
f = plt.figure()
#path = "/home/esteban/Escritorio/Practica/Plots/G13_millennium" # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Plots/G14_millennium" # Gonzalez2014 
path = "/home/esteban/Escritorio/Practica/Plots/L16_millennium" # Lacey2016 
os.chdir( path )

plt.title(r"$n = 10^{%.1f} /h^{-3} Mpc^3$" %rho)
plt.plot(bins,logxi,'k.')
plt.ylim(-2,6)
plt.xlabel("$log(d/h^{-1}Mpc)$")
plt.ylabel(r"$log(\xi)$")
#plt.legend(frameon=False)
plt.minorticks_on()
f.savefig('xi_03.pdf')

In [10]:
#path = "/home/esteban/Escritorio/Practica/Data/G13_millennium" # Guo2013 
#path = "/home/esteban/Escritorio/Practica/Data/G14_millennium" # Gonzalez2014 
path = "/home/esteban/Escritorio/Practica/Data/L16_millennium" # Lacey2016 
os.chdir( path )
np.savetxt("xi_data_03.txt", np.array([bins, logxi]).T)